In [41]:
import pandas as pd
import math 
import numpy as np
from scipy import stats
from obspy.geodetics import degrees2kilometers

In [2]:
def init_dataframe(path):
    df = pd.read_csv(path, sep=',')
    return df

In [3]:
ref1 = init_dataframe('data/ref1.csv')
ref2 = init_dataframe('data/ref2.csv')

In [4]:
ref2.head(4)

,LAT,LON,GSPEED,CRS,NLAT,NLON,NCRS
0,48.050451,17.569713,0.82,209.35145,48.050450,17.569712,208.75295
1,48.050450,17.569712,0.89,208.75295,48.050450,17.569712,208.97508
2,48.050450,17.569712,0.93,208.97508,48.050449,17.569711,209.55402
3,48.050449,17.569711,0.93,209.55402,48.050448,17.569710,208.77161


In [5]:
lap = init_dataframe('data/laps/lap1-0.csv')
lap.head(2)

,LAT,LON,GSPEED,CRS,NLAT,NLON,NCRS
0,48.049231,17.567851,0.00,0.0,48.049231,17.567849,0.0
1,48.049231,17.567849,0.04,0.0,48.049231,17.567849,0.0


In [6]:
lap[['LAT','LON']].head(4)

,LAT,LON
0,48.049231,17.567851
1,48.049231,17.567849
2,48.049231,17.567849
3,48.049229,17.567850


In [38]:
def line_length(x1,y1,x2,y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def find_closest_point(point, lap):
    minIndex = 0
    minLength = math.inf
    for i in lap.index:
        lat = lap.loc[i].LAT
        lon = lap.loc[i].LON

        length = line_length(lat, lon, point.LAT, point.LON)
        if length < minLength:
            minIndex = i
            minLength = length

    return minIndex

def find_angle_between_vectors(vector_A, vector_B):
    unit_vector_A = vector_A / np.linalg.norm(vector_A) 
    unit_vector_B = vector_B / np.linalg.norm(vector_B)
    dot_product = np.dot(unit_vector_A, unit_vector_B)
    return np.arccos(dot_product)                       #return angle in radians 

def create_vector(point_A, point_B):
    return [point_B.LAT - point_A.LAT, point_B.LON - point_A.LON]

def shortest_distance(x1, y1, a, b, c):  
    perpendicular = abs((a * x1 + b * y1 + c)) / (math.sqrt(a ** 2 + b ** 2)) 
    return perpendicular

def find_shortest_distance(point1, point2, point3):
        x = [point2.LAT, point3.LAT]
        y = [point2.LON, point3.LON]
        slope , intercept,_,_,_ = stats.linregress(x, y)
        return shortest_distance(point1.LAT, point1.LON, slope, -1, intercept)

In [ ]:
def line_length(x1,y1,x2,y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [9]:
x = [10,15]
y = [20,30]
slope, intercept, _, _, _ = stats.linregress(x,y)
print(slope, intercept)

2.0 0.0


In [29]:
x1 = 10
y1 = 20
a = 2
b = -1
c = 0
shortest_distance(x1, y1, a, b, c)   

Perpendicular distance is 0.0


In [45]:
sum_of_distances = 0
for i in lap.index:
    point = lap.loc[i]

    closest_index = find_closest_point(point, ref1)
    closest_point = ref1.loc[closest_index]

    neighbor_i = len(ref1)-1 if closest_index == 0 else closest_index-1
    neighbor1 = ref1.loc[neighbor_i]
    neighbor_i = 0 if len(ref1) == closest_index+1 else closest_index+1
    neighbor2 = ref1.loc[neighbor_i]

    v1 = create_vector(closest_point, point)
    v2 = create_vector(closest_point, neighbor1)
    v3 = create_vector(closest_point, neighbor2)

    angle1 = find_angle_between_vectors(v1,v2)
    angle2 = find_angle_between_vectors(v1,v3)

    degrees90 = math.pi/2
    min_dist = -1
    if angle1 > degrees90 and angle2 > degrees90:
        min_dist = line_length(point.LAT, point.LON, closest_point.LAT, closest_point.LON)
    elif angle1 < degrees90 and angle2 < degrees90:
        dist1 = find_shortest_distance(point, closest_point, neighbor1)
        dist2 = find_shortest_distance(point, closest_point, neighbor2)
        min_dist = dist1 if dist1 <= dist2 else dist2
    elif angle1 < degrees90:
        min_dist = find_shortest_distance(point, closest_point, neighbor1)
    elif angle2 < degrees90:
        min_dist = find_shortest_distance(point, closest_point, neighbor2)

    if min_dist == -1:
        print ('ERROR: Could not find distance')       
        print("Indices: {} {}\nAngles: {} {}".format(i, closest_index, angle1, angle2))
    elif math.isnan(min_dist):
        print("NAN value!!!\nIndices: {} {}\nAngles: {} {}".format(i, closest_index, angle1, angle2))
    elif min_dist < 0:
        print("Negative value!!!\nIndices: {} {}\nAngles: {} {}".format(i, closest_index, angle1, angle2))
    else:
        min_dist = degrees2kilometers(min_dist) * 100000    # in centimeters
        sum_of_distances += min_dist        
    
print ("Sum is {}cm".format(sum_of_distances))
print ("Average distance from reference lap is {}cm".format(sum_of_distances/len(lap)))

with open('average_perpendiculars.txt', 'a') as file:
    file.write('{} cm\n'.format(sum_of_distances/len(lap)))


    

Sum is nancm
Average distance from reference lap is nancm
